<a href="https://colab.research.google.com/github/Andicleomj/Machine-Learning/blob/main/Week10/Regression%20Model%3A%20MLP%20Regression/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Andi Cleopatra Maryam Jamila
#### 1103213071
#### Regression Model: MLP Regression


In [1]:
# Install dan import library
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm

In [6]:
# Load dataset
iris = load_iris()
X = iris.data
y = iris.data[:, 0]

In [4]:
# Preprocessing data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Konversi ke tensor PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [8]:
# Dataset dan DataLoader
class IrisDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

train_dataset = IrisDataset(X_train, y_train)
test_dataset = IrisDataset(X_test, y_test)

In [9]:
# Model MLP
class MLPRegression(nn.Module):
    def __init__(self, input_dim, hidden_layers, activation_function):
        super(MLPRegression, self).__init__()
        layers = []
        current_dim = input_dim
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(current_dim, hidden_dim))
            layers.append(activation_function)
            current_dim = hidden_dim
        layers.append(nn.Linear(current_dim, 1))  # Output layer
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [10]:
# Grid Hyperparameter
hidden_layer_configurations = [[4], [8, 16], [32, 64, 32]]
activation_functions = {
    "ReLU": nn.ReLU(),
    "Sigmoid": nn.Sigmoid(),
    "Tanh": nn.Tanh(),
    "Linear": nn.Identity()
}
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
epochs_list = [1, 10, 25, 50, 100, 250]
batch_sizes = [16, 32, 64, 128, 256, 512]

In [16]:
# Loop eksperimen dengan itertools.product
hyperparameter_combinations = itertools.product(
    hidden_layer_configurations,  # Kombinasi hidden layers
    activation_functions.keys(),  # Nama fungsi aktivasi
    learning_rates,               # Daftar learning rate
    epochs_list,                  # Daftar epoch
    batch_sizes                   # Daftar batch size
)

In [17]:
# Total kombinasi
total_combinations = (
    len(hidden_layer_configurations)
    * len(activation_functions)
    * len(learning_rates)
    * len(epochs_list)
    * len(batch_sizes)
)

In [23]:
# List untuk menyimpan hasil eksperimen
results = []

In [24]:
# Iterasi kombinasi hyperparameter dengan tqdm
for hidden_layers, activation_name, learning_rate, epochs, batch_size in tqdm(
    hyperparameter_combinations,
    total=total_combinations
):
    # DataLoader untuk training dan testing
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Inisialisasi model, loss function, dan optimizer
    model = MLPRegression(input_dim=4, hidden_layers=hidden_layers, activation_function=activation_functions[activation_name])
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Proses training
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()

    # Evaluasi model
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)
        test_loss = criterion(y_pred, y_test).item()

    # Menyimpan hasil eksperimen
    results.append({
        "Hidden Layers": hidden_layers,
        "Activation Function": activation_name,
        "Learning Rate": learning_rate,
        "Epochs": epochs,
        "Batch Size": batch_size,
        "Test Loss": test_loss
    })

100%|█████████▉| 2591/2592 [16:31<00:00,  2.61it/s]


In [25]:
# Konversi hasil eksperimen ke DataFrame
results_df = pd.DataFrame(results)
print(results_df.sort_values(by="Test Loss").head())

     Hidden Layers Activation Function  Learning Rate  Epochs  Batch Size  \
1643       [8, 16]              Linear           0.01     100          16   
751            [4]              Linear           0.10     250          64   
2478  [32, 64, 32]              Linear           0.10     250          32   
1614       [8, 16]              Linear           0.10     250          32   
2515  [32, 64, 32]              Linear           0.01     250          64   

         Test Loss  
1643  6.063298e-14  
751   6.821211e-14  
2478  7.579123e-14  
1614  7.579123e-14  
2515  7.579123e-14  
